## Import of libraries

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings,openai.OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.chains import RetrievalQA


In [28]:


# Specify the path to your parquet file
file_path = 'train-00010-of-00071-7c89af7fb68feb34.parquet'

# Read the parquet file into a DataFrame
df = pd.read_parquet(file_path)

# Display the first few rows
df.head()


,image,text
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,<html>\n<style>\n* {\n box-sizing: border-box...
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,<html>\n<style>\nbody {\n display: flex;\n f...
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,<html>\n<style>\nbody {\n font-family: Aria...
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,<html>\n<style>\nbody {\n font-family: Aria...
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,<html>\n<style>\n:root {\n --primary: #c0ffee...


In [32]:
df = df.iloc[:,-1]

In [33]:


# The Ollama instance with the base URL and model name
ollama = OllamaEmbeddings(base_url='http://localhost:11434', model="mistral")

# creating embeddings for ollama
doc_embeddings = ollama.embed_documents(df)

In [36]:
# Save the embeddings to a text file
with open('embedding_doc.txt', 'w') as f:
    for embedding in doc_embeddings:
        embedding_str = ' '.join(map(str, embedding))
        f.write(embedding_str + '\n')

In [51]:

df = df.to_frame(name='text')


In [53]:



documents = [Document(page_content=text) for text in df]


In [55]:
# Assuming 'text' is the column with the text data
text_series = df['text']

In [56]:


# Convert each row of the DataFrame into a Document object
documents = [
    Document(page_content=row['text'], metadata=row.drop('text').to_dict())
    for index, row in df.iterrows()
]

In [57]:


persist_directory = 'vec_dir'

# Create the vector store with Chroma
vector_db = Chroma.from_documents(
    documents=documents, 
    embedding=ollama, 
    persist_directory=persist_directory
)


In [58]:
retriever = vector_db.as_retriever()

In [59]:

llm = Ollama(model="mistral")


chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True
)


## Retrival using RAG and querying using the llm

In [ ]:
query = "html css js code for social media login page"
response = chain({"query": query})


answer = response['result']

print("Answer:", answer)